In [9]:
import boto3
import time
import uuid
from boto3.dynamodb.conditions import Key

# For a Boto3 client.
boto3.setup_default_session(profile_name='td-prod')

dynamodb = boto3.resource("dynamodb")
table = dynamodb.Table('CoreStack-CoreTable97EB8292-OSGHIVMEFPBR')


In [5]:
# tom-remoteca user_id 
toms_id = "b5af90ee-bf35-4a66-a4ed-9ee7ee0591e6"
company_id = "company_exampleid1234567"
example_query = {
    "summary": "adfasdfa",
    "date_submitted": "1692922541",
    "attachments": [],
    "query_id": "query_9c89aa70-888d-4280-b670-db96c7303a8e",
    "submittor_email": "tom@remoteca.co.uk",
    "company_id": "company_exampleid1234567",
    "usecase_tags": [
        "IFRS 3 - Business Combinations"
    ],
    "jurisdiction": "asdf",
    "query_status": "Submitted",
    "query_title": "NotificationTestNewQ",
    "advice_type": "Technical Accounting Advice",
    "company_name": "Example Company",
    "is_public": "false",
    "id": "query_9c89aa70-888d-4280-b670-db96c7303a8e",
    "submittor_id": "b5af90ee-bf35-4a66-a4ed-9ee7ee0591e6"
}

In [10]:
def create_notification(company_id, query, badge):
    notification_id = f"notification_{uuid.uuid4()}"

    primary_keys = {
        "PK": f"COMPANY#{company_id}#USER#{query['submittor_id']}",
        "SK": f"NOTIFICATION#{notification_id}",
    }

    notification = {
        "id": notification_id,
        "is_public": query['is_public'],
        "date": int(time.time() * 1000),
        "badge": badge,
        "query_title": query['query_title'],
        'query_id': query['id']
    }

    res = table.put_item(
        Item={
            **primary_keys,
            "notification": notification
        }
    )
    return notification

In [11]:
create_notification(company_id=company_id, query=example_query, badge="Ready to nibble")

{'id': 'notification_b56f1b48-945c-4219-b1b4-bd3040398fb6',
 'is_public': 'false',
 'date': 1692923648828,
 'badge': 'Ready to nibble',
 'query_title': 'NotificationTestNewQ',
 'query_id': 'query_9c89aa70-888d-4280-b670-db96c7303a8e'}

In [26]:
activity_1 = {"type": "comment", "comment":"lemons", "from": "jeff"}
activity_2 = {"type": "comment", "comment":"asdfawf", "from": "31dff"}
import ulid

def create_query_activity(query_id, activity):
    activity_id = f"activity_{str(ulid.new())}"
    
    activity['id'] = activity_id
    activity['query_id'] = query_id
    
    res = table.put_item(
        Item={
            'PK': f"QUERY#{query_id}",
            'SK': f"ACTIVITY#{activity_id}",
            'activity': activity
        }
    )
    return activity

create_query_activity(QUERY_ID, activity_2)

{'type': 'comment',
 'comment': 'asdfawf',
 'from': '31dff',
 'id': 'activity_01H4TS4Q72GPX414SFXD37X38Z',
 'query_id': 'query_f888990e-1606-410f-aeb7-85262fef7d5e'}

In [30]:
def get_activity(query_id):
    res = table.query(
        KeyConditionExpression=Key('PK').eq(f"QUERY#{query_id}") & Key('SK').begins_with(f"ACTIVITY#")
    )
    return [i['activity'] for i in res.get('Items', [])]
get_activity(QUERY_ID)
    
    

[{'comment': 'lemons',
  'from': 'jeff',
  'id': 'activity_01H4TQ69N7M2342MMGKNBCTSTN',
  'type': 'comment',
  'query_id': 'query_f888990e-1606-410f-aeb7-85262fef7d5e'},
 {'comment': 'lemons',
  'from': 'jeff',
  'id': 'activity_01H4TQ6R36QC3H28S1DAYWQ5Q7',
  'type': 'comment',
  'query_id': 'query_f888990e-1606-410f-aeb7-85262fef7d5e'},
 {'comment': 'asdfawf',
  'from': '31dff',
  'id': 'activity_01H4TQ7569HGXKRP6F8149D8SG',
  'type': 'comment',
  'query_id': 'query_f888990e-1606-410f-aeb7-85262fef7d5e'},
 {'comment': 'asdfawf',
  'from': '31dff',
  'id': 'activity_01H4TS4Q72GPX414SFXD37X38Z',
  'type': 'comment',
  'query_id': 'query_f888990e-1606-410f-aeb7-85262fef7d5e'}]